<a href="https://colab.research.google.com/github/tttequila/Kaggle_20Q/blob/main/LLM_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuring your kaggle token, see more details in the [**Configure your API key**](https://ai.google.dev/gemma/docs/setup) section

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%%bash
mkdir ~/.kaggle
# change the first path to your path of kaggle.json
cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content'

In [2]:
%%bash
pip install -U torch immutabledict sentencepiece

# pip install -q -U kerasnlp
# pip install -q -U keras>=3

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1

### Loading mode

In [4]:
# %%bash
!git clone https://github.com/google/gemma_pytorch.git

# !mkdir /kaggle/working/gemma/
# !mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [5]:
# login the kaggle (need to store you kaggle.json to your google dirve)
import kagglehub
kagglehub.login()

In [6]:
# Choose variant and machine type
VARIANT = '7b-it-quant'
MACHINE_TYPE = 'cuda'

import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'
# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

Attaching model 'google/gemma/pyTorch/7b-it-quant' to your Colab notebook...


In [ ]:
# import sys
# sys.path.append("/content/gemma_pytorch/gemma")
# print(sys.path)
# from gemma.config import get_config_for_7b, get_config_for_2b
# from gemma.model import GemmaForCausalLM, GemmaModel
# import contextlib, torch


In [11]:
import sys
sys.path.append("gemma_pytorch/gemma")
sys.path.append("gemma_pytorch")
print(sys.path)
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM, GemmaModel
import contextlib, torch


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/gemma_pytorch/gemma', 'gemma_pytorch/gemma', 'gemma_pytorch/gemma', 'gemma_pytorch', 'gemma_pytorch/gemma', 'gemma_pytorch']


In [12]:

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

## Loading model

In [13]:
import torch
import gemma

# Set up model config.
model_config = get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT
# model_config.architecture = gemma.config.Architecture.GEMMA_2

with _set_default_tensor_type(model_config.get_dtype()):
  device = torch.device(MACHINE_TYPE)
  model = GemmaForCausalLM(model_config)
  model.load_weights(ckpt_path)
  model = model.to(device).eval()

## Play with Gemma

### format your prompts

In [16]:
# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='What is a good place for travel in the US?'
    )
)
print('Chat prompt:\n', prompt)

Chat prompt:
 <start_of_turn>user
What is a good place for travel in the US?<end_of_turn>



### Chat & Response

In [18]:
import time
# generate with formatted prompts
start = time.time()
response_1 = model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=100,
)



[response in 44.94276571273804]
The best place to travel in the US depends on your interests and preferences. Here are a few recommendations based on different categories:

**For first-time visitors:**

* **Grand Canyon National Park, Arizona:** A bucket-list destination with stunning natural beauty and a variety of hiking trails.
* **San Francisco, California:** A vibrant city with a rich history, Golden Gate Bridge, Alcatraz Island, and a variety of cultural attractions.
* **Chicago, Illinois:** A friendly city


In [27]:
# or directly talk to Gemma
start = time.time()
response_2 = model.generate(
    'Write a poem about an llm writing a poem.',
    device=device,
    output_len=100,
)
print(f"[response in {(time.time()-start):.2f}s]\n{response_2}")

[response in 44.85s]


A pen in hand, a heart aflame,
A mind ablaze in a poetic flame.
The muse whispers secrets sweet,
And the ink flows, a lyrical feat.

The llm's hand dances across the page,
With passion and grace, a rhythmic rage.
Words dance and leap in a dance of rhyme,
A melody of love, sorrow, and time.

The inkwell reflects the llm's soul,
A canvas painted to reveal a
